<a href="https://colab.research.google.com/github/falarion08/CEPARCO_MP1/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Group 3
Authors:

Barrion, Ryan Onil

Geronimo, Michael Robert

Roledo, Jan Carlo

Solis, Frances Danielle

---
# C++ Version of Dot Product

In [ ]:
%%writefile c_dotp.c
#include <bits/stdc++.h>
#include <ctime>
using namespace std;


//C++ Dot Product Function
void dot_product (int n, float h_out, float* h_in1, float* h_in2){
  for (int i = 0; i < n; i++)
   h_out += h_in1[i] * h_in2[i];
}

int main() {
    const unsigned int ARRAY_SIZE = 1<<24;
    const unsigned ARRRAY_BYTES = ARRAY_SIZE * sizeof(float);
    const unsigned int run = 30;

    float *in1, *in2, out;
    in1 = (float*)malloc(ARRRAY_BYTES);
    in2 = (float*)malloc(ARRRAY_BYTES);

    clock_t start, end;

    //init data
    for(int i = 0; i <ARRAY_SIZE;i++){
        in1[i]=float(i);
        in2[i]=float(i);
    }
      
    dot_product(ARRAY_SIZE, out,in1,in2);

    start = clock();
    for(int i = 0; i<run; i++){
      dot_product(ARRAY_SIZE,out,in1,in2);
    }
    end = clock();
    double time_taken = (((double)(end-start))* 1e6/CLOCKS_PER_SEC)/(double)run;
    cout<< "C++ function took an average of "  << time_taken << " microseconds for array size " <<  ARRAY_SIZE;
  
    //Check for errors
    
    
   
   //free memory
   free(in1);
   free(in2);

   return 0;

}

Writing c_dotp.c


In [ ]:
%%shell
g++ c_dotp.c -o c_dotp

In [ ]:
%%shell
./c_dotp

C++ function took an average of 59066 microseconds for array size 16777216

___
#2. CUDA Without Cooperative Group Dot Product

In [18]:
%%writefile cuda_uncooperative_threads.cu

#include <stdio.h>
#include <stdlib.h>

__global__
void dotProduct(float *x,float *y, double *dotp,long unsigned int ARRAY_SIZE){

  extern __shared__ double mul_result[];
  mul_result[threadIdx.x]  = 0.0;


  int index = blockIdx.x * gridDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;


  for (int i = index; i < ARRAY_SIZE; i += stride){
      mul_result[threadIdx.x] += (double)y[i] * x[i];
  }
  __syncthreads();

  if(threadIdx.x == 0){
    double sum = 0.0;

    for(int i = 0 ; i < blockDim.x; ++i){
      sum = sum + mul_result[i];
    }
    atomicAdd(dotp, sum);
  }

}
int main(void){
  // Declare Number of Elements
  const unsigned long int ARRAY_SIZE = 1<<28;
  const unsigned long int ARRAY_BYTES = ARRAY_SIZE * sizeof(ARRAY_SIZE); 

  // Declare Dimensions
  const int nBlocks = 512;
  const int nThreads = nBlocks;

  int device = -1;
  cudaGetDevice(&device);

  float *x, *y;
  double *dotp;

  // Allocate resources for variables x and y;
  cudaMallocManaged(&x,ARRAY_BYTES);
  cudaMallocManaged(&y, ARRAY_BYTES);

  // Allocate only a single space to store the result of sum
  cudaMallocManaged(&dotp, sizeof(double));
  *dotp = 0.0;
  
  //Prefetch Part 1
  cudaMemAdvise(x,ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
  cudaMemAdvise(y,ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);

  cudaMemPrefetchAsync(dotp,sizeof(double), device, NULL);

  // Initialize Variables
  for (int i = 0; i < ARRAY_SIZE; ++i){
    x[i] = (float)((i + 1) % 50);
    y[i]  = x[i] + 2.0;
  }


  // Prefetch Part 2
  cudaMemAdvise(x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);
  cudaMemAdvise(y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);

  cudaMemAdvise(x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemAdvise(y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

  cudaMemPrefetchAsync(x,ARRAY_BYTES, device, NULL);
  cudaMemPrefetchAsync(y,ARRAY_BYTES, device, NULL);

  // Initial launch to prevent additional time for cache miss

  //Launch Kernel
  dotProduct<<<nBlocks,nThreads, sizeof(double) * nThreads>>>(x,y,dotp,ARRAY_SIZE);
  cudaDeviceSynchronize();
  
  // Prefetch Result
  cudaMemPrefetchAsync(dotp, sizeof(double), cudaCpuDeviceId, NULL);

  // Check for Error by measuring percent error
  double expected = 0;

  for(int i = 0; i < ARRAY_SIZE; ++i){
    expected = expected + (double)(x[i] * y[i]);
  }

  float pError = abs((dotp[0]-expected)/expected) * 100;

  printf("\n\nNumber of Elements: %lu\n",ARRAY_SIZE);
  printf("Number of Blocks: %d\n",nBlocks);
  printf("Number of Threads: %d\n",nThreads);

  printf("\n\nExpected:%f\nComputed:%f\n\n", dotp[0], expected );
  printf("Percent Error: %f\n\n",pError );

  cudaFree(x);
  cudaFree(y);
  cudaFree(dotp);

  return 0;
}

Overwriting cuda_uncooperative_threads.cu


In [19]:
%%shell

nvcc cuda_uncooperative_threads.cu -o cuda_uncooperative_threads -arch=sm_60 

nvprof ./cuda_uncooperative_threads


==2924== NVPROF is profiling process 2924, command: ./cuda_uncooperative_threads


Number of Elements: 268435456
Number of Blocks: 512
Number of Threads: 512


Expected:230183398508.000000
Computed:230183398508.000000

Percent Error: 0.000000

==2924== Profiling application: ./cuda_uncooperative_threads
==2924== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  9.9210ms         1  9.9210ms  9.9210ms  9.9210ms  dotProduct(float*, float*, double*, unsigned long)
      API calls:   26.08%  166.34ms         3  55.446ms  3.1857ms  82.708ms  cudaFree
                   22.81%  145.47ms         3  48.490ms  18.967us  145.40ms  cudaMallocManaged
                   18.39%  117.29ms         1  117.29ms  117.29ms  117.29ms  cudaDeviceSynchronize
                   16.58%  105.77ms         6  17.629ms  2.0800us  55.256ms  cudaMemAdvise
                   16.11%  102.77ms         4  25.694ms  36.608us  102.38ms  cudaMemPre

---
#3. CUDA Cooperative Group Dot Product

In [ ]:
%%writefile cuda_group.cu

#include <stdio.h>
#include <math.h>
#include <cooperative_groups.h>

using namespace cooperative_groups;
namespace cg = cooperative_groups;

#define BLOCK_SIZE 1024

//device "sub-function" to fill shared memory and add elements in parallel
__device__
double reduction_dotprod(thread_block g, float *temp, float val_1, float val_2){

  //get thread index
  int lane = g.thread_rank();

  val_1 = val_1 * val_2;
  g.sync();

  //"Grid-Stride" loop
  for (int i = blockDim.x / 2; i > 0; i /= 2){        
    temp[lane] = val_1;
    g.sync();
    if (lane < i) val_1 += temp[lane + i];
    g.sync();
  }

  return val_1;
}

__global__
void dotprod_group(float *in_x, float *in_y, double *out){

  //create shared memory space
  __shared__ float temp[BLOCK_SIZE];

  //initialize thread block for binary-reduction sum
  auto g = this_thread_block();

  //assign thread with initial value from input vector
  unsigned int i = blockIdx.x * blockDim.x + threadIdx.x;

  //perform reduction sum
  double block_sum = reduction_dotprod(g, temp, in_x[i], in_y[i]);

  //the first thread (thread 0) will have the final result
  if (threadIdx.x == 0){
    atomicAdd(out, block_sum);
  }

}



int main() {

  const unsigned int ARRAY_SIZE = 1<<24;

  const unsigned int ARRAY_BYTES = ARRAY_SIZE * sizeof(int);

  //size of shared bytes in sum_group() kernel
  const unsigned int SHARED_BYTES = BLOCK_SIZE * sizeof(int);

  const int NUM_BLOCKS = (ARRAY_SIZE + BLOCK_SIZE - 1) / BLOCK_SIZE;
  printf("numElem = %d\tnumThreads = %d\tnumBlocks = %d\n", ARRAY_SIZE, BLOCK_SIZE, NUM_BLOCKS);

  //declare ARRAY_BYTES or allocate in GPU
  float *in_x, *in_y;
  double *out;
  cudaMallocManaged(&in_x, ARRAY_BYTES);
  cudaMallocManaged(&in_y, ARRAY_BYTES);
  cudaMallocManaged(&out, sizeof(double));

  //prefetch input data to CPU
  cudaMemPrefetchAsync(in_x, ARRAY_BYTES, cudaCpuDeviceId, NULL);
  cudaMemPrefetchAsync(in_y, ARRAY_BYTES, cudaCpuDeviceId, NULL);

  //init data
  for(int i = 0; i < ARRAY_SIZE; i++)
    in_x[i] = in_y[i] = 0.000001f * i;

  int device = -1;
  cudaGetDevice(&device); // get device number of GPU

  cudaMemPrefetchAsync(out, ARRAY_BYTES, device, NULL);

  //Prefetch and Memory Advise input data to device
  cudaMemAdvise(in_x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);
  cudaMemPrefetchAsync(in_x, ARRAY_BYTES, device, NULL);

  cudaMemAdvise(in_y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, device);
  cudaMemPrefetchAsync(in_y, ARRAY_BYTES, device, NULL);

  cudaMemPrefetchAsync(out, sizeof(double), device, NULL);


  //perform dot product with Cooperative Groups (thread_block)
  dotprod_group<<<NUM_BLOCKS, BLOCK_SIZE, SHARED_BYTES>>> (in_x, in_y, out);

  cudaDeviceSynchronize();


  //prefetch input data to CPU
  cudaMemAdvise(in_x, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemPrefetchAsync(in_x, ARRAY_BYTES, cudaCpuDeviceId, NULL);

  cudaMemAdvise(in_y, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemPrefetchAsync(in_y, ARRAY_BYTES, cudaCpuDeviceId, NULL);

  cudaMemAdvise(out, sizeof(float), cudaMemAdviseSetReadMostly, cudaCpuDeviceId);
  cudaMemPrefetchAsync(out, sizeof(float), cudaCpuDeviceId, NULL);

  //check for errors
  //NOTE: Floating point arithmetic is NOT ASSOCIATIVE
  //todo: Match(?) adding order of algorithm
  //may not actually be possible; blocks are always executed in any order
  //and can't (as far as I'm aware) be set manually.
  double sum = 0;
  for(int i = 0; i < ARRAY_SIZE/2; i++) {
    sum += (in_x[i] * in_y[i] + in_x[i+ARRAY_SIZE/2] * in_y[i+ARRAY_SIZE/2]);
  }

  double sum_seq = 0;
  for (int i = 0; i < ARRAY_SIZE; i++){
    sum_seq += in_x[i] * in_y[i];
  }
  printf("Sequential Sum: %lf\t", sum_seq);
  printf("'Binary Split' Sum: %lf\t Coop-Group Reduction Sum: %lf\n",sum, *out);

  double diff = abs((*out - sum) / sum) * 100.0;

  printf("Difference with split sum: %.5f%%\n", diff);

  //free memory in GPU
  cudaFree(in_x);
  cudaFree(in_y);
  cudaFree(out);

  return 0;
}

Writing cuda_group.cu


In [ ]:
%%shell
nvcc cuda_group.cu -o cuda_group -arch=sm_60

In [ ]:
%%shell
nvprof ./cuda_group

numElem = 16777216	numThreads = 1024	numBlocks = 16384
==695== NVPROF is profiling process 695, command: ./cuda_group
Sequential Sum: 1574122012.273521	'Binary Split' Sum: 1574122012.255829	 Coop-Group Reduction Sum: 1574122011.782480
Difference with split sum: 0.00000%
==695== Profiling application: ./cuda_group
==695== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.7659ms         1  2.7659ms  2.7659ms  2.7659ms  dotprod_group(float*, float*, double*)
      API calls:   78.54%  233.94ms         3  77.981ms  17.951us  233.88ms  cudaMallocManaged
                   14.14%  42.109ms         9  4.6787ms  7.4060us  17.234ms  cudaMemPrefetchAsync
                    2.99%  8.9173ms         3  2.9724ms  99.315us  4.4375ms  cudaFree
                    2.28%  6.7977ms         5  1.3595ms  1.5290us  3.4656ms  cudaMemAdvise
                    1.57%  4.6651ms         1  4.6651ms  4.6651ms  4.6651ms  cudaDeviceSync